In [6]:
from methods import *
import random
import matplotlib.pyplot as plt
import os

# 0 - Choose the protein structure you want to study

In [7]:
#structure_id = "1nd1" 
#file_name = "DATA//" + structure_id + ".pdb"
#protein = True
structure_id = "4ldb"
file_name = "DATA//" + structure_id + ".pdb"
protein = True
#structure_id = "guanine"  #The code can be different if it is a molecule or a protein
#file_name = "DATA//" + structure_id + ".pdb"
#protein = False
#structure_id = "4ogs"
#file_name = "DATA//" + structure_id + ".pdb"
#protein = True
#structure_id = "4pyi"
#file_name = "DATA//" + structure_id + ".pdb"
#protein = True
#structure_id = "6tpk"
#file_name = "DATA//" + structure_id + ".pdb"
#protein = True

random.seed(27112000)

### 1 - Read the .pdb file and plot the 3D structure

In [8]:
if protein:
    structure = PDB_prot_reader(structure_id, file_name)
    print("Total number of carbon atoms: ", np.shape(structure[1])[0])
else:
    structure = PDB_mol_reader(structure_id, file_name)
    print("Total number of atoms: ", np.shape(structure[1])[0])

if os.path.exists("RESULTS//" + structure_id + ".gif"):
    print("Verify that this script has not already been run with this structure")
else:
    plot_3d(structure_id, structure[1])

Total number of carbon atoms:  987


### 2 - Analyse the 3D structure to determine the different values of epsilon

In [9]:
min = [np.min(idx) for idx in zip(*structure[1])]
max = [np.max(idx) for idx in zip(*structure[1])]

random_pair = [[random.randint(0, np.shape(structure[1])[0]-1), random.randint(0, np.shape(structure[1])[0]-1)]  for i in range(1000)]
dist = np.sort(np.array([np.linalg.norm(structure[1][idx[0]] - structure[1][idx[1]]) for idx in random_pair]))

while dist[0] == 0:
    dist = dist[1:]

min_dist, max_dist = np.min(dist), np.max(dist)
index = np.linspace(0, 900, 10, dtype = int)

""" if min_dist < 4:
    epsilon = [min_dist * 1.2, min_dist * 1.5, min_dist*2]
elif min_dist >= 4 and min_dist < 10:
    epsilon = [min_dist * 1.2, min_dist * 1.5, min_dist*2]
else:
    epsilon = [min_dist * 1.2, min_dist * 1.5, min_dist*2] """

epsilons = [4, 5, 6, 7, 8]

print(" Min (x,y,z): ", min,'\n',
    "Max (x,y,z): ", max, '\n',
    "Min euclidean distance: ", min_dist, '\n',
    "Max euclidean distance: ", max_dist, '\n',
    "Epsilon range: ", epsilons, '\n'
    " Distance batch: ", dist[index])

 Min (x,y,z):  [-52.497, -87.507, -38.126] 
 Max (x,y,z):  [35.074, -28.036, 53.7] 
 Min euclidean distance:  3.1416407 
 Max euclidean distance:  90.0552 
 Epsilon range:  [4, 5, 6, 7, 8] 
 Distance batch:  [ 3.1416407 18.24382   25.072935  30.905268  36.014294  40.706802
 45.21195   50.06922   55.484085  65.063866 ]


In [10]:
rips_complex = []
simplex_tree = []
total_inc1 = []
total_inc2 = []

for epsilon in epsilons:

    rips = generate_rips(structure[1], epsilon)
    edges = np.array([s[0] for s in rips.get_skeleton(1) if len(s[0])==2])
    triangles = np.array([s[0] for s in rips.get_skeleton(2) if len(s[0])==3])

    local, inc1, inc2 = inconsistency(structure, rips, edges, triangles)

    total_inc1.append(inc1)
    total_inc1.append(inc2)

    if os.path.exists("RESULTS//Rips_" + "_" + structure_id + "_" + str(epsilon) + ".gif"):
        print("Verify that this script has not already been run with this structure")
    else:
        plot_rips(structure[1], rips, edges, triangles, structure_id, epsilon, local)

    print(inc1)
    print(inc2)

Rips complex is of dimension 1 - 1955 simplices - 987 vertices.
2.1482694617931524e-10
1.0664936399003045e-11
Rips complex is of dimension 2 - 3194 simplices - 987 vertices.
865.2470291357201
28.424517909009587
Rips complex is of dimension 2 - 5271 simplices - 987 vertices.
1219.3508465756206
32.800453393430026
Rips complex is of dimension 2 - 8876 simplices - 987 vertices.
2065.878202174794
42.51652571163447
Rips complex is of dimension 2 - 13835 simplices - 987 vertices.
2679.789960607619
47.81085920614698
